# Preprocessing using Beam/Dataflow

## Import libraries

In [1]:
import sys
import os
import pathlib
import logging
import subprocess
import datetime
import subprocess, requests
import apache_beam as beam
from google.cloud import bigquery
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import StandardOptions
from apache_beam.options.pipeline_options import GoogleCloudOptions

## Defined GCP env variables

In [2]:
try:
    tmp=os.environ['PROJECT_ID']
except:
    print('Env variable PROJECT not defined!') 

try:
    tmp=os.environ['BUCKET_NAME']
except:
    print('Env variable BUCKET_NAME not defined!') 

try:
    tmp=os.environ['REGION']
except:
    print('Env variable REGION not defined!') 

try:    
    tmp=os.environ['GOOGLE_APPLICATION_CREDENTIALS']
except:
    print('Env variable GOOGLE_APPLICATION_CREDENTIALS not defined!') 

try:
    tmp=os.environ['REQUESTS_CA_BUNDLE']
except:
    print('Env variable REQUESTS_CA_BUNDLE not defined!') 

try:
    tmp=os.environ['AXA_CH_CA_BUNDLE']
except:
    print('Env variable AXA_CA_CA_BUNDLE not defined!') 

Env variable GOOGLE_APPLICATION_CREDENTIALS not defined!
Env variable REQUESTS_CA_BUNDLE not defined!
Env variable AXA_CA_CA_BUNDLE not defined!


## Preprocessing using Beam/Dataflow

### Define a query to get data

In [3]:
# define query table
def create_query():
    query = """
    SELECT
    *
    FROM
    `bigquery-public-data.stackoverflow.tags`
    LIMIT 100
    """

    return query

### Get the schema of the input table

In [4]:
# getting schema
! bq show --format=prettyjson bigquery-public-data:stackoverflow.tags 

{
  "creationTime": "1477491432274", 
  "etag": "+9nuQ6pxbVUXEEgkOowfuw==", 
  "id": "bigquery-public-data:stackoverflow.tags", 
  "kind": "bigquery#table", 
  "lastModifiedTime": "1575026613006", 
  "location": "US", 
  "numBytes": "2284573", 
  "numLongTermBytes": "0", 
  "numRows": "55665", 
  "schema": {
    "fields": [
      {
        "mode": "NULLABLE", 
        "name": "id", 
        "type": "INTEGER"
      }, 
      {
        "mode": "NULLABLE", 
        "name": "tag_name", 
        "type": "STRING"
      }, 
      {
        "mode": "NULLABLE", 
        "name": "count", 
        "type": "INTEGER"
      }, 
      {
        "mode": "NULLABLE", 
        "name": "excerpt_post_id", 
        "type": "INTEGER"
      }, 
      {
        "mode": "NULLABLE", 
        "name": "wiki_post_id", 
        "type": "INTEGER"
      }
    ]
  }, 
  "selfLink": "https://bigquery.googleapis.com/bigquery/v2/projects/bigquery-public-data/datasets/stackoverflow/tables/tags", 
  "tableReference": {
    

In [5]:
table_schema = {
    'fields': [
        {
            'mode': 'NULLABLE',
            'name': 'id',
            'type': 'INTEGER'
        },
        {
            'mode': 'NULLABLE',
            'name': 'tag_name',
            'type': 'STRING'
        },
        {
            'mode': 'NULLABLE',
            'name': 'count',
            'type': 'INTEGER'
        },
        {
            'mode': 'NULLABLE',
            'name': 'excerpt_post_id',
            'type': 'INTEGER'
        },
        {
            'mode': 'NULLABLE',
            'name': 'wiki_post_id',
            'type': 'INTEGER'
        }
    ]
}

### Define the preprocessing steps

In [6]:
def preprocess():
    """
    Arguments:
        -RUNNER: "DirectRunner" or "DataflowRunner". Specfy to run the pipeline locally or on Google Cloud respectively.
    Side-effects:
        -Creates and executes dataflow pipeline.
        See https://beam.apache.org/documentation/programming-guide/#creating-a-pipeline
    """
    job_name = 'test-stackoverflow' + '-' + datetime.datetime.now().strftime('%y%m%d-%H%M%S')
    project = os.environ['PROJECT_ID']
    region = os.environ['REGION']
    output_dir = "gs://{0}/stackoverflow/".format(os.environ['BUCKET_NAME'])

    # options
    options = PipelineOptions()
    google_cloud_options = options.view_as(GoogleCloudOptions)
    google_cloud_options.project =  project
    google_cloud_options.job_name =  job_name
    google_cloud_options.region = region
    google_cloud_options.staging_location = os.path.join(output_dir, 'tmp', 'staging')
    google_cloud_options.temp_location = os.path.join(output_dir, 'tmp')
    # done by command line
    options.view_as(StandardOptions).runner = 'DataflowRunner'

    # instantantiate Pipeline object using PipelineOptions
    print('Launching Dataflow job {} ... hang on'.format(job_name))

    p = beam.Pipeline(options=options)
    output = p | 'Read from BigQuery' >> beam.io.Read(beam.io.BigQuerySource(
        # query
        query=create_query(),
        # use standard SQL for the above query
        use_standard_sql=True))
    output | 'Write to BigQuery' >> beam.io.WriteToBigQuery(
        # The table name is a required argument for the BigQuery
        table='test_stackoverflow_beam',
        dataset='test',
        project=project,
        # Here we use the JSON schema read in from a JSON file.
        # Specifying the schema allows the API to create the table correctly if it does not yet exist.
        schema=table_schema,
        # Creates the table in BigQuery if it does not yet exist.
        create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED,
        # Deletes all data in the BigQuery table before writing.
        write_disposition=beam.io.BigQueryDisposition.WRITE_TRUNCATE)
        # not needed, from with clause

    if options.view_as(StandardOptions).runner == 'DataflowRunner':
        print('DataflowRunner')
        p.run()
    else:
        print('Default: DirectRunner')
        result = p.run()
        result.wait_until_finish()
    print('Done')

In [7]:
if __name__ == '__main__':
    logging.getLogger().setLevel(logging.DEBUG)

    print('Starting main process ...')
    preprocess()

Starting main process ...
Launching Dataflow job test-stackoverflow-191129-165617 ... hang on
DataflowRunner


/home/.conda-env/env_nlp_text_class/lib/python3.6/site-packages/apache_beam/runners/dataflow/dataflow_runner.py:696: BeamDeprecationWarning: BigQuerySink is deprecated since 2.11.0. Use WriteToBigQuery instead.
  kms_key=transform.kms_key))
INFO:root:Setting socket default timeout to 60 seconds.
INFO:root:socket default timeout is 60.0econds.
DEBUG:root:Connecting using Google Application Default Credentials.
INFO:root:Starting GCS upload to gs://nlp-text-classification/stackoverflow/tmp/staging/test-stackoverflow-191129-165617.1575046577.861365/pipeline.pb...
INFO:oauth2client.transport:Attempting refresh to obtain initial access_token
INFO:oauth2client.transport:Attempting refresh to obtain initial access_token
INFO:root:Completed GCS upload to gs://nlp-text-classification/stackoverflow/tmp/staging/test-stackoverflow-191129-165617.1575046577.861365/pipeline.pb in 0 seconds.
INFO:root:Downloading source distribution of the SDK from PyPi
INFO:root:Executing command: ['/home/.conda-env/

Done
